In [1]:
%%capture
!pip install hummingbird-ml

In [2]:
import pickle
from os import listdir
from os.path import isfile, join
import numpy as np

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from hummingbird.ml import convert,load

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('drive/MyDrive/RealLifeViolencePickle/training_violence2.pickle', 'rb') as handle:
    training_run2 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/training_violence3.pickle', 'rb') as handle:
    training_run3 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/training_violence4.pickle', 'rb') as handle:
    training_run4 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/training_violence_test.pickle', 'rb') as handle:
    allTestingData = pickle.load(handle)

In [5]:
allTrainingData = np.concatenate([training_run2, training_run3, training_run4])

In [6]:
allTrainingData.shape

(73384, 3780)

In [7]:
allTestingData.shape

(22918, 3780)

In [8]:
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence2.pickle', 'rb') as labels:
  labels_run2 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence3.pickle', 'rb') as labels:
  labels_run3 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence4.pickle', 'rb') as labels:
  labels_run4 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence_test.pickle', 'rb') as labels:
  listOfFrameLabelsTest = pickle.load(labels)

In [9]:
listOfFrameLabels = np.concatenate([labels_run2, labels_run3, labels_run4])

## SVM (Support Vector Machine)

In [10]:
#framesTrain, framesTest, labelsTrain, labelsTest = train_test_split(allTrainingData, listOfFrameLabels, test_size= 0.00)
framesTrain, labelsTrain = allTrainingData, listOfFrameLabels
framesTest, labelsTest = allTestingData, listOfFrameLabelsTest
svcClassifier = LinearSVC(random_state=0,max_iter=1000,C=0.2)
svcClassifier.fit(framesTrain,labelsTrain)
labelPredict = svcClassifier.predict(framesTest)
print("---------------SVM---------------")
print(confusion_matrix(labelsTest, labelPredict))  
print(classification_report(labelsTest, labelPredict)) 
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


---------------SVM---------------
[[10671   185]
 [ 3587  8475]]
              precision    recall  f1-score   support

           0       0.75      0.98      0.85     10856
           1       0.98      0.70      0.82     12062

    accuracy                           0.84     22918
   macro avg       0.86      0.84      0.83     22918
weighted avg       0.87      0.84      0.83     22918

Total Accuracy:  0.8354132123221921


Measuring inference time

In [11]:
svc_time = %timeit -o svcClassifier.predict(framesTest)

10 loops, best of 5: 149 ms per loop


In [19]:
model_pytorch = convert(svcClassifier, 'torch')
# Timing the inference for Pytorch on CPU only
pred_cpu = %timeit -o model_pytorch.predict(framesTest)

1 loop, best of 5: 209 ms per loop


In [24]:
model_pytorch.to('cuda')
pred_gpu = %timeit -o model_pytorch.predict(framesTest)

1 loop, best of 5: 269 ms per loop


## KNN (K-Nearest Neighbour)

In [ ]:
print("---------------KNN---------------")
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(framesTrain, labelsTrain)
labelPredict = neigh.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

---------------KNN---------------
[[10529  2833]
 [  546  7600]]
              precision    recall  f1-score   support

           0       0.95      0.79      0.86     13362
           1       0.73      0.93      0.82      8146

    accuracy                           0.84     21508
   macro avg       0.84      0.86      0.84     21508
weighted avg       0.87      0.84      0.85     21508

Total Accuracy:  0.8428956667286591


In [ ]:
neigh_time = %timeit -o neigh.predict(framesTest)

## Logistic Regression

In [ ]:
print("---------------Logistic Regression---------------")
logreg = LogisticRegression()
logreg.fit(framesTrain, labelsTrain)
labelPredict = logreg.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

---------------Logistic Regression---------------


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[[10417  2945]
 [  149  7997]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.87     13362
           1       0.73      0.98      0.84      8146

    accuracy                           0.86     21508
   macro avg       0.86      0.88      0.85     21508
weighted avg       0.89      0.86      0.86     21508

Total Accuracy:  0.8561465501208853


In [ ]:
logreg_time = %timeit -o logreg.predict(framesTest)

## Decision Tree

In [ ]:
print("---------------Decision Tree---------------")
clf_model = DecisionTreeClassifier(criterion="gini", random_state=42,max_depth=3, min_samples_leaf=5)   
clf_model.fit(framesTrain, labelsTrain)
labelPredict = clf_model.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

---------------Decision Tree---------------
[[10806  2556]
 [  354  7792]]
              precision    recall  f1-score   support

           0       0.97      0.81      0.88     13362
           1       0.75      0.96      0.84      8146

    accuracy                           0.86     21508
   macro avg       0.86      0.88      0.86     21508
weighted avg       0.89      0.86      0.87     21508

Total Accuracy:  0.8647015064162172


In [ ]:
logreg_time = %timeit -o logreg.predict(framesTest)

In [ ]:
model_pytorch = convert(logreg, 'torch')
# Timing the inference for Pytorch on CPU only
pred_cpu = %timeit -o model_pytorch.predict(X_test)

In [ ]:
%%capture 
model_pytorch.to('cuda')
pred_gpu = %timeit -o model_pytorch.predict(X_test)

## LDA (Linear Discriminant Analysis)

In [ ]:
print("---------------LDA---------------")
clf = LinearDiscriminantAnalysis()
clf.fit(framesTrain, labelsTrain)
labelPredict = clf.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

---------------LDA---------------


## Gradient Boosting

In [31]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_features=2, max_depth=2, random_state=0)
gb_clf.fit(framesTrain, labelsTrain)
labelPredict = gb_clf.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

[[10856     0]
 [  659 11403]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     10856
           1       1.00      0.95      0.97     12062

    accuracy                           0.97     22918
   macro avg       0.97      0.97      0.97     22918
weighted avg       0.97      0.97      0.97     22918

Total Accuracy:  0.9712453093638188


In [32]:
gb_time = %timeit -o gb_clf.predict(framesTest)

1 loop, best of 5: 464 ms per loop


In [34]:
model_pytorch = convert(gb_clf, 'torch')
# Timing the inference for Pytorch on CPU only
pred_cpu = %timeit -o model_pytorch.predict(framesTest)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


1 loop, best of 5: 405 ms per loop


In [35]:
model_pytorch.to('cuda')
pred_gpu = %timeit -o model_pytorch.predict(framesTest)

1 loop, best of 5: 224 ms per loop


## Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(framesTrain, labelsTrain)
labelPredict = clf.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

[[10854     2]
 [ 1162 10900]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     10856
           1       1.00      0.90      0.95     12062

    accuracy                           0.95     22918
   macro avg       0.95      0.95      0.95     22918
weighted avg       0.95      0.95      0.95     22918

Total Accuracy:  0.9492102277685662


In [26]:
clf_time = %timeit -o clf.predict(framesTest)

1 loop, best of 5: 1.94 s per loop


In [27]:
model_pytorch = convert(clf, 'torch')
# Timing the inference for Pytorch on CPU only
pred_cpu = %timeit -o model_pytorch.predict(framesTest)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


1 loop, best of 5: 11.3 s per loop


In [29]:
model_pytorch.to('cuda')
pred_gpu = %timeit -o model_pytorch.predict(framesTest)

1 loop, best of 5: 765 ms per loop
